# CycleGAN Implementation in tensorflow v2

In [3]:
import os
import glob
import time
import argparse

from PIL import Image
from datetime import datetime

import tensorflow as tf

import utils
import models

# Convert Data to TFRecords

Define the parameters for the GAN

In [4]:
LEARNING_RATE = 0.0002
BETA_1 = 0.5
LAMBDA_CYCLE = 10.0
LAMBDA_ID = 5
POOL_SIZE = 50
EPOCHS = 40
DECAY_EPOCHS = 100
SHUFFLE_SIZE = 10000
BATCH_SIZE = 2
DATASET = 'monet2photo'

In [8]:
# Read each image using PIL and write it to a tf.Example instance
def convert_to_tfexample(img_path):
    try:
        with open(img_path, 'rb') as f:
            content = f.read()
        with Image.open(img_path) as img:
            img.load()
            if img.format != 'JPEG':
                print('Image format error. Path: ' + img_path + '\tFormat: ' + img.format)
            assert (img.format == 'JPEG')
            filename = os.path.basename(img_path)
            example = tf.train.Example(
                        features=tf.train.Features(
                        feature={
                        'image/encoded': tf.train.Feature(bytes_list=tf.train.BytesList(value=[content])),
                        'image/format': tf.train.Feature(bytes_list=tf.train.BytesList(value=['JPEG'.encode()])),
                        'image/width': tf.train.Feature(int64_list=tf.train.Int64List(value=[img.width])),
                        'image/height': tf.train.Feature(int64_list=tf.train.Int64List(value=[img.height])),
                        'image/filename': tf.train.Feature(bytes_list=tf.train.BytesList(value=[filename.encode()]))
                        }))
            return example
    except Exception as e:
        print(e)
        return None

In [9]:
# Helper function to iterate files, convert and write
def tf_convert_write(input_files, output_record):
    with tf.io.TFRecordWriter(output_record) as tfwriter:
        for f in input_files:
            example = convert_to_tfexample(f)
            tfwriter.write(example.SerializeToString())
    print('Converted to tfrecord: ' + output_record)

In [12]:
# Convert raw image data into tfrecords to use with TFX later
def convert_data(datafolder, dataset):
    '''
    Expected directory structure under path is 
    trainA, trainB, testA, testB which the GAN will train on
    '''
    train_A_files = glob.glob(os.path.join(datafolder, dataset, 'trainA', '*'))
    train_B_files = glob.glob(os.path.join(datafolder, dataset, 'trainB', '*'))
    test_A_files = glob.glob(os.path.join(datafolder, dataset, 'testA', '*'))
    test_B_files = glob.glob(os.path.join(datafolder, dataset, 'testB', '*'))
    #input_files = [train_A_files, train_B_files]
    
    tf_record_dir = os.path.join(os.getcwd(), "tfdata", DATASET)
    if not os.path.exists(tf_record_dir):
        os.makedirs(tf_record_dir)
        
    train_A_outfile = os.path.join(tf_record_dir,'trainA.tfrecord')
    train_B_outfile = os.path.join(tf_record_dir,'trainB.tfrecord')
    test_A_outfile = os.path.join(tf_record_dir,'testA.tfrecord')
    test_B_outfile = os.path.join(tf_record_dir,'testB.tfrecord')
   
    tf_convert_write(train_A_files, train_A_outfile)
    tf_convert_write(train_B_files, train_B_outfile)
    tf_convert_write(test_A_files, test_A_outfile)
    tf_convert_write(test_B_files, test_B_outfile)
    print("Finished converting data to TFRecords in directory: " + tf_record_dir)

In [13]:
datafolder = "datasets"
convert_data(datafolder, DATASET)

Converted to tfrecord: D:\Workdir\DL_Project\tfdata\monet2photo\trainA.tfrecord
Converted to tfrecord: D:\Workdir\DL_Project\tfdata\monet2photo\trainB.tfrecord
Converted to tfrecord: D:\Workdir\DL_Project\tfdata\monet2photo\testA.tfrecord
Converted to tfrecord: D:\Workdir\DL_Project\tfdata\monet2photo\testB.tfrecord
Finished converting data to TFRecords in directory: D:\Workdir\DL_Project\tfdata\monet2photo


In [17]:
# Read TFRecrods and pre-process training images
def process_dataset(filepath):
    raw_dataset = tf.data.TFRecordDataset(filepath)

    image_feature_description = {
        'image/height': tf.io.FixedLenFeature([], tf.int64),
        'image/width': tf.io.FixedLenFeature([], tf.int64),
        'image/format': tf.io.FixedLenFeature([], tf.string),
        'image/encoded': tf.io.FixedLenFeature([], tf.string),
    }

    def normalize_images(example_proto):
        # Parse the input tf.Example proto using the dictionary above.
        features = tf.io.parse_single_example(example_proto, image_feature_description)
        encoded_img = features['image/encoded']
        
        img = tf.image.decode_jpeg(encoded_img, 3)
        
        img = tf.image.random_flip_left_right(img)
        img = tf.image.resize(img, [286, 286])
        img = tf.image.random_crop(img, [256, 256, tf.shape(img)[-1]])
        img = img / 127.5 - 1  # normalize from 1 to -1
        return img

    norm_dataset = raw_dataset.map(normalize_images)
    return norm_dataset

In [18]:
train_a = process_dataset('tfdata/{}/trainA.tfrecord'.format(DATASET))
train_b = process_dataset('tfdata/{}/trainB.tfrecord'.format(DATASET))
combined_dataset = tf.data.Dataset.zip((train_a, train_b)).shuffle(SHUFFLE_SIZE).batch(int(BATCH_SIZE))

total_batches = 0
for _ in combined_dataset:
    total_batches += 1
#total_batches = count_dataset_batches(combined_dataset)
print('Batch size: {}, Total batches per epoch: {}'.format(BATCH_SIZE, total_batches))

Batch size: 2, Total batches per epoch: 536


# Define the model

Based on https://arxiv.org/pdf/1703.10593.pdf and https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix/blob/master/models/cycle_gan_model.py

**See models.py** for the implementation of the cycleGAN model. 

References (some tf2, mostly pytorch): 

https://hardikbansal.github.io/CycleGANBlog/

https://github.com/LynnHo/CycleGAN-Tensorflow-2

https://github.com/jason9693/cycleGAN-tensorflow-v2

In [19]:
A_B_gen = models.make_generator_model(n_blocks=9)
B_A_gen = models.make_generator_model(n_blocks=9)
discriminator_b = models.make_discriminator_model()
discriminator_a = models.make_discriminator_model()
gen_lr_scheduler = utils.LinearDecay(LEARNING_RATE, EPOCHS * total_batches, DECAY_EPOCHS * total_batches)
dis_lr_scheduler = utils.LinearDecay(LEARNING_RATE, EPOCHS * total_batches, DECAY_EPOCHS * total_batches)

# Checkpoint Mgr

Use the ADAM optimizer from Keras with a Linear Rate Decay. Used in a pytorch model I was following. See utils.py for reference code and source

In [32]:
gen_lr_scheduler.get_config()

{'initial_learning_rate': 0.0002, 'total_steps': 21440, 'step_decay': 53600}

In [33]:
dis_lr_scheduler.get_config()

{'initial_learning_rate': 0.0002, 'total_steps': 21440, 'step_decay': 53600}

In [20]:
optimizer_gen = tf.keras.optimizers.Adam(gen_lr_scheduler, BETA_1)
optimizer_dis = tf.keras.optimizers.Adam(dis_lr_scheduler, BETA_1)

Create a checkpint manager that will save the training checkpoints. Since GANs take a really long time to train -- want to make best use of resources

In [21]:
checkpoint_dir = './checkpoints-{}'.format(DATASET)
checkpoint = tf.train.Checkpoint(
    A_B_gen=A_B_gen,
    B_A_gen=B_A_gen,
    discriminator_b=discriminator_b,
    discriminator_a=discriminator_a,
    optimizer_gen=optimizer_gen,
    optimizer_dis=optimizer_dis,
    epoch=tf.Variable(0))

Checkpoint manager which restores the latest checkpoing using a nice high level API call

In [22]:
checkpoint_manager = tf.train.CheckpointManager(
    checkpoint, checkpoint_dir, max_to_keep=None)
checkpoint.restore(checkpoint_manager.latest_checkpoint)
if checkpoint_manager.latest_checkpoint:
    print("Restored from {}".format(checkpoint_manager.latest_checkpoint))
else:
    print("Initializing from scratch.")

Restored from ./checkpoints-monet2photo\ckpt-19


# Define the loss variables

Using keras metrics variable instances that will be utilized in training and creating tensorboard logs

In [23]:
# Total Generator loss for both
gen_loss_total = tf.keras.metrics.Mean('gen_loss_total', dtype=tf.float32)
# Total Discriminator Loss
dis_loss_total = tf.keras.metrics.Mean('dis_loss_total', dtype=tf.float32)
# Cycle loss a
a2b2a_cycle_loss = tf.keras.metrics.Mean('a2b2a_cycle_loss', dtype=tf.float32) # todo: change a2b2a
b2a2b_cycle_loss = tf.keras.metrics.Mean('b2a2b_cycle_loss', dtype=tf.float32)
gen_a2b_loss = tf.keras.metrics.Mean('gen_a2b_loss', dtype=tf.float32)
gen_b2a_loss = tf.keras.metrics.Mean('gen_b2a_loss', dtype=tf.float32)
dis_b_loss = tf.keras.metrics.Mean('dis_b_loss', dtype=tf.float32)
dis_a_loss = tf.keras.metrics.Mean('dis_a_loss', dtype=tf.float32)
id_b2a_loss = tf.keras.metrics.Mean('id_b2a_loss', dtype=tf.float32)
id_a2b_loss = tf.keras.metrics.Mean('id_a2b_loss', dtype=tf.float32)

Add loss functions and a two way iamge pool class. Refernce implementation in utils.py

ImagePool class is an image buffer that stores previously generated images. This class helps in updating the two discriminator using the entire history of generated images rather than the one currently produced by the generator. The outcome is that the discriminators retains the context better in terms of edges and other features since the training data is all unpaired images.

In [24]:
mse_loss = tf.keras.losses.MeanSquaredError()
mae_loss = tf.keras.losses.MeanAbsoluteError()
fake_pool_b2a = utils.ImagePool(POOL_SIZE)
fake_pool_a2b = utils.ImagePool(POOL_SIZE)

Helper function to write all training metrics to logs as training proceeds

In [25]:
def write_metrics(epoch):
    with train_summary_writer.as_default():
        tf.summary.scalar('gen_a2b_loss', gen_a2b_loss.result(), step=epoch)
        tf.summary.scalar('gen_b2a_loss', gen_b2a_loss.result(), step=epoch)
        tf.summary.scalar('dis_b_loss', dis_b_loss.result(), step=epoch)
        tf.summary.scalar('dis_a_loss', dis_a_loss.result(), step=epoch)
        tf.summary.scalar('id_a2b_loss', id_a2b_loss.result(), step=epoch)
        tf.summary.scalar('id_b2a_loss', id_b2a_loss.result(), step=epoch)
        tf.summary.scalar('gen_loss_total', gen_loss_total.result(), step=epoch)
        tf.summary.scalar('dis_loss_total', dis_loss_total.result(), step=epoch)
        tf.summary.scalar('A_B_A_cycle_loss',a2b2a_cycle_loss.result(), step=epoch)
        tf.summary.scalar('B_A_B_cycle_loss', b2a2b_cycle_loss.result(), step=epoch)
        tf.summary.scalar('gen_learning_rate', gen_lr_scheduler.current_learning_rate, step=epoch)
        tf.summary.scalar('dis_learning_rate', dis_lr_scheduler.current_learning_rate, step=epoch)

    gen_a2b_loss.reset_states()
    gen_b2a_loss.reset_states()
    dis_b_loss.reset_states()
    dis_a_loss.reset_states()
    id_b2a_loss.reset_states()
    id_a2b_loss.reset_states()
    return

Return ones or zeros for the loss based on whether the instance of the image passed in the real or generated

In [35]:
def calc_gan_loss(prediction, real_img):
    if real_img:
        return mse_loss(prediction, tf.ones_like(prediction))
    else:
        return mse_loss(prediction, tf.zeros_like(prediction))


Using the tf.function decorator because it converts the training to a graph representation, resulting in performance boost. Reference: https://pgaleone.eu/tensorflow/tf.function/2019/03/21/dissecting-tf-function-part-1/

In [27]:
@tf.function
def train_generator(images_a, images_b):
    real_imgs_a = images_a
    real_imgs_b = images_b

    with tf.GradientTape() as tape:
        # Cycle from A to B to A
        A_B_fake = A_B_gen(real_imgs_a, training=True)
        B_A_rec = B_A_gen(A_B_fake, training=True)
        # Cycle from B to A to B
        B_A_fake = B_A_gen(real_imgs_b, training=True)
        A_B_rec = A_B_gen(B_A_fake, training=True)

        # use real image to generate a real image -- their identity should be preserved (think horse to horse)
        A_B_id = A_B_gen(real_imgs_b, training=True)
        B_A_id = B_A_gen(real_imgs_a, training=True)
        loss_A_B_id = mae_loss(A_B_id, real_imgs_b) # cycle loss representing the fidelity of reconstructed img
        loss_B_A_id = mae_loss(B_A_id, real_imgs_a)

        # A to B generator attempts to trick Discriminator B that the generated image is B
        A_B_gen_loss = calc_gan_loss(discriminator_b(A_B_fake, training=True), True)
        
        # Generator B2A tries to trick Discriminator A that the generated image is A
        B_A_gen_loss = calc_gan_loss(discriminator_a(B_A_fake, training=True), True)
        
        A_B_A_cycle_loss = mae_loss(B_A_rec, real_imgs_a) # Identity losses to ensure no overtraining
        B_A_B_cycle_loss = mae_loss(A_B_rec, real_imgs_b)

        # Total generator loss is equal to all individual losses (cycle losses multiplied by # of cycles and identifying loss by # identify cycles or tries)
        loss_gen_total = A_B_gen_loss + B_A_gen_loss + (A_B_A_cycle_loss + B_A_B_cycle_loss) * LAMBDA_CYCLE + (loss_A_B_id + loss_B_A_id) * LAMBDA_ID

    train_vars = A_B_gen.trainable_variables + B_A_gen.trainable_variables
    gradient_gen = tape.gradient(loss_gen_total, train_vars)
    optimizer_gen.apply_gradients(zip(gradient_gen, train_vars))

    # Metrics
    a2b2a_cycle_loss(A_B_A_cycle_loss)
    b2a2b_cycle_loss(B_A_B_cycle_loss)
    gen_loss_total(loss_gen_total)
    gen_a2b_loss(A_B_gen_loss)
    gen_b2a_loss(B_A_gen_loss)
    gen_b2a_loss(loss_B_A_id)
    gen_a2b_loss(loss_A_B_id)

    loss_dict = {
        'loss_gen_a2b': A_B_gen_loss,
        'loss_gen_b2a': B_A_gen_loss,
        'loss_id_a2b': loss_A_B_id,
        'loss_id_b2a': loss_B_A_id,
        'A_B_A_cycle_loss': A_B_A_cycle_loss,
        'B_A_B_cycle_loss': B_A_B_cycle_loss,
        'loss_gen_total': loss_gen_total,
    }
    return A_B_fake, B_A_fake, loss_dict



In [28]:
@tf.function
def train_discriminator(images_a, images_b, A_B_fake, B_A_fake):
    real_a = images_a
    real_imgs_b = images_b

    with tf.GradientTape() as tape:

        # Discriminator A should classify real_a as A
        loss_gan_dis_a_real = calc_gan_loss(discriminator_a(real_a, training=True), True)
        
        # Discriminator A should classify generated B_A_fake as not A
        loss_gan_dis_a_fake = calc_gan_loss(discriminator_a(B_A_fake, training=True), False)

        # Discriminator B should classify real_imgs_b as B
        loss_gan_dis_b_real = calc_gan_loss(discriminator_b(real_imgs_b, training=True), True)
        
        # Discriminator B should classify generated A_B_fake as not B
        loss_gan_dis_b_fake = calc_gan_loss(discriminator_b(A_B_fake, training=True), False)

        # Total discriminator loss
        loss_dis_a = (loss_gan_dis_a_real + loss_gan_dis_a_fake) * 0.5
        loss_dis_b = (loss_gan_dis_b_real + loss_gan_dis_b_fake) * 0.5
        loss_dis_total = loss_dis_a + loss_dis_b

    trainable_variables = discriminator_a.trainable_variables + discriminator_b.trainable_variables
    gradient_dis = tape.gradient(loss_dis_total, trainable_variables)
    optimizer_dis.apply_gradients(zip(gradient_dis, trainable_variables))

    # Metrics
    dis_a_loss(loss_dis_a)
    dis_b_loss(loss_dis_b)
    dis_loss_total(loss_dis_total)

    return {
        'loss_dis_b': loss_dis_b,
        'loss_dis_a': loss_dis_a,
        'loss_dis_total': loss_dis_total
    }

In [29]:
# todo: defined
def train_step(images_a, images_b, epoch, step):
    A_B_fake, B_A_fake, gen_loss_dict = train_generator(images_a, images_b)

    B_A_fake_from_pool = fake_pool_b2a.query(B_A_fake)
    A_B_fake_from_pool = fake_pool_a2b.query(A_B_fake)

    dis_loss_dict = train_discriminator(images_a, images_b, A_B_fake_from_pool, B_A_fake_from_pool)

    gen_loss_list = ['{}:{} '.format(k, v) for k, v in gen_loss_dict.items()]
    dis_loss_list = ['{}:{} '.format(k, v) for k, v in dis_loss_dict.items()]

    tf.print('Epoch {} Step {} '.format(epoch, step), ' '.join(gen_loss_list + dis_loss_list))

In [30]:
current_time = datetime.now().strftime("%Y%m%d-%H%M%S")
train_log_dir = 'logs/{}/{}/train'.format(DATASET, current_time)
train_summary_writer = tf.summary.create_file_writer(train_log_dir)

# Training Loop

In [31]:
# todo: defined
def train(data_set, num_epoch):
    for _epoch in range(checkpoint.epoch + 1, num_epoch + 1):
        start = time.time()
        print('Starting epoch {}. Learning rate: {}, {}'.format( epoch, 
            gen_lr_scheduler.current_learning_rate, dis_lr_scheduler.current_learning_rate))
        
        # Train
        for (_step, _batch) in enumerate(data_set):
            train_step(_batch[0], _batch[1], epoch, _step)
            
        write_metrics(_epoch)
        
        checkpoint.epoch.assign_add(1)
        if epoch %2 == 0:
            save_path = checkpoint_manager.save()
            print("Saved checkpoint for epoch {}: {}".format(int(checkpoint.epoch), save_path))
            print('Time for epoch {} is {} sec'.format(epoch, time.time() - start))

In [56]:
train(combined_dataset, EPOCHS)
print('Finished training.')

Starting epoch 39. Learning rate: <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=0.0002>, <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=0.0002>
Epoch 39 Step 0  loss_gen_a2b:0.6702579259872437  loss_gen_b2a:0.07966842502355576  loss_id_a2b:0.12855015695095062  loss_id_b2a:0.08416980504989624  A_B_A_cycle_loss:0.08065621554851532  B_A_B_cycle_loss:0.13700540363788605  loss_gen_total:3.990142345428467  loss_dis_b:0.18025146424770355  loss_dis_a:0.3314996361732483  loss_dis_total:0.511751115322113 
Epoch 39 Step 1  loss_gen_a2b:0.5527761578559875  loss_gen_b2a:0.2550315856933594  loss_id_a2b:0.09307940304279327  loss_id_b2a:0.11707022786140442  A_B_A_cycle_loss:0.1302591860294342  B_A_B_cycle_loss:0.07678477466106415  loss_gen_total:3.928995370864868  loss_dis_b:0.1339893639087677  loss_dis_a:0.19612301886081696  loss_dis_total:0.33011239767074585 
Epoch 39 Step 2  loss_gen_a2b:0.32532361149787903  loss_gen_b2a:0.40900519490242004  loss_id_a2b:0.0909818485379219  loss_i

Epoch 39 Step 23  loss_gen_a2b:0.5376220941543579  loss_gen_b2a:0.8509014844894409  loss_id_a2b:0.063633993268013  loss_id_b2a:0.1694662719964981  A_B_A_cycle_loss:0.20521074533462524  B_A_B_cycle_loss:0.0996163934469223  loss_gen_total:5.602296829223633  loss_dis_b:0.05892384052276611  loss_dis_a:0.03479223698377609  loss_dis_total:0.0937160775065422 
Epoch 39 Step 24  loss_gen_a2b:0.2645703852176666  loss_gen_b2a:0.1490822583436966  loss_id_a2b:0.10740914195775986  loss_id_b2a:0.09107714891433716  A_B_A_cycle_loss:0.0937737226486206  B_A_B_cycle_loss:0.09919507801532745  loss_gen_total:3.3357720375061035  loss_dis_b:0.4008200168609619  loss_dis_a:0.45540642738342285  loss_dis_total:0.8562264442443848 
Epoch 39 Step 25  loss_gen_a2b:0.6602970957756042  loss_gen_b2a:0.3619401156902313  loss_id_a2b:0.06621704995632172  loss_id_b2a:0.09390032291412354  A_B_A_cycle_loss:0.10355542600154877  B_A_B_cycle_loss:0.08047781884670258  loss_gen_total:3.663156747817993  loss_dis_b:0.06307625770568

Epoch 39 Step 46  loss_gen_a2b:0.29201117157936096  loss_gen_b2a:0.21467728912830353  loss_id_a2b:0.11605703085660934  loss_id_b2a:0.18478882312774658  A_B_A_cycle_loss:0.12242914736270905  B_A_B_cycle_loss:0.12543535232543945  loss_gen_total:4.48956298828125  loss_dis_b:0.18444669246673584  loss_dis_a:0.3195655941963196  loss_dis_total:0.5040122866630554 
Epoch 39 Step 47  loss_gen_a2b:0.2079942524433136  loss_gen_b2a:0.4260537028312683  loss_id_a2b:0.09011977910995483  loss_id_b2a:0.07498575747013092  A_B_A_cycle_loss:0.08625971525907516  B_A_B_cycle_loss:0.11446910351514816  loss_gen_total:3.4668638706207275  loss_dis_b:0.19969983398914337  loss_dis_a:0.04531370475888252  loss_dis_total:0.2450135350227356 
Epoch 39 Step 48  loss_gen_a2b:0.4615744650363922  loss_gen_b2a:0.14158064126968384  loss_id_a2b:0.06298026442527771  loss_id_b2a:0.10881894826889038  A_B_A_cycle_loss:0.13008660078048706  B_A_B_cycle_loss:0.05866958945989609  loss_gen_total:3.3497133255004883  loss_dis_b:0.265012

Epoch 39 Step 69  loss_gen_a2b:0.8558555841445923  loss_gen_b2a:0.5004297494888306  loss_id_a2b:0.07954662293195724  loss_id_b2a:0.08427198976278305  A_B_A_cycle_loss:0.08231642842292786  B_A_B_cycle_loss:0.07882644981145859  loss_gen_total:3.7868072986602783  loss_dis_b:0.37625953555107117  loss_dis_a:0.15818652510643005  loss_dis_total:0.5344460606575012 
Epoch 39 Step 70  loss_gen_a2b:0.705345094203949  loss_gen_b2a:0.15848013758659363  loss_id_a2b:0.1451910436153412  loss_id_b2a:0.08303577452898026  A_B_A_cycle_loss:0.112235888838768  B_A_B_cycle_loss:0.1479474902153015  loss_gen_total:4.606793403625488  loss_dis_b:0.1908087134361267  loss_dis_a:0.08734740316867828  loss_dis_total:0.2781561017036438 
Epoch 39 Step 71  loss_gen_a2b:0.5474417805671692  loss_gen_b2a:0.3003610074520111  loss_id_a2b:0.08206315338611603  loss_id_b2a:0.0905570462346077  A_B_A_cycle_loss:0.08534470200538635  B_A_B_cycle_loss:0.0667785257101059  loss_gen_total:3.2321360111236572  loss_dis_b:0.07230185717344

Epoch 39 Step 92  loss_gen_a2b:0.42248648405075073  loss_gen_b2a:0.5113292932510376  loss_id_a2b:0.1005832701921463  loss_id_b2a:0.12099676579236984  A_B_A_cycle_loss:0.11173790693283081  B_A_B_cycle_loss:0.08715862035751343  loss_gen_total:4.030681133270264  loss_dis_b:0.09504293650388718  loss_dis_a:0.22316968441009521  loss_dis_total:0.318212628364563 
Epoch 39 Step 93  loss_gen_a2b:0.35563674569129944  loss_gen_b2a:0.46685048937797546  loss_id_a2b:0.06795751303434372  loss_id_b2a:0.06558853387832642  A_B_A_cycle_loss:0.07964703440666199  B_A_B_cycle_loss:0.07639848440885544  loss_gen_total:3.050673007965088  loss_dis_b:0.1854282021522522  loss_dis_a:0.22584748268127441  loss_dis_total:0.4112756848335266 
Epoch 39 Step 94  loss_gen_a2b:0.09905603528022766  loss_gen_b2a:0.3353358805179596  loss_id_a2b:0.12086285650730133  loss_id_b2a:0.13368995487689972  A_B_A_cycle_loss:0.11879881471395493  B_A_B_cycle_loss:0.08933069556951523  loss_gen_total:3.7884511947631836  loss_dis_b:0.3451566

...

Epoch 39 Step 460  loss_gen_a2b:0.12243479490280151  loss_gen_b2a:0.2756861448287964  loss_id_a2b:0.05741236358880997  loss_id_b2a:0.0779939591884613  A_B_A_cycle_loss:0.07748212665319443  B_A_B_cycle_loss:0.07649506628513336  loss_gen_total:2.614924192428589  loss_dis_b:0.2570192813873291  loss_dis_a:0.23023678362369537  loss_dis_total:0.4872560501098633 
Epoch 39 Step 461  loss_gen_a2b:0.23320314288139343  loss_gen_b2a:0.3678189814090729  loss_id_a2b:0.16196823120117188  loss_id_b2a:0.07781407237052917  A_B_A_cycle_loss:0.08745770901441574  B_A_B_cycle_loss:0.1274804174900055  loss_gen_total:3.949315071105957  loss_dis_b:0.22320887446403503  loss_dis_a:0.08797728270292282  loss_dis_total:0.31118616461753845 
Epoch 39 Step 462  loss_gen_a2b:0.3398003578186035  loss_gen_b2a:0.16530156135559082  loss_id_a2b:0.10555297136306763  loss_id_b2a:0.07847575843334198  A_B_A_cycle_loss:0.10250188410282135  B_A_B_cycle_loss:0.11328837275505066  loss_gen_total:3.5831480026245117  loss_dis_b:0.1256

Epoch 39 Step 483  loss_gen_a2b:0.6156034469604492  loss_gen_b2a:0.18669432401657104  loss_id_a2b:0.1687280237674713  loss_id_b2a:0.11246512830257416  A_B_A_cycle_loss:0.10135741531848907  B_A_B_cycle_loss:0.15624502301216125  loss_gen_total:4.784287929534912  loss_dis_b:0.11689500510692596  loss_dis_a:0.11687897145748138  loss_dis_total:0.23377397656440735 
Epoch 39 Step 484  loss_gen_a2b:0.23070736229419708  loss_gen_b2a:0.09488768875598907  loss_id_a2b:0.12058009952306747  loss_id_b2a:0.16239723563194275  A_B_A_cycle_loss:0.13425350189208984  B_A_B_cycle_loss:0.10484183579683304  loss_gen_total:4.131435394287109  loss_dis_b:0.17882080376148224  loss_dis_a:0.14080853760242462  loss_dis_total:0.31962934136390686 
Epoch 39 Step 485  loss_gen_a2b:0.33524057269096375  loss_gen_b2a:0.018470363691449165  loss_id_a2b:0.08186426758766174  loss_id_b2a:0.08067784458398819  A_B_A_cycle_loss:0.08332923799753189  B_A_B_cycle_loss:0.08583620190620422  loss_gen_total:2.8580758571624756  loss_dis_b:

Epoch 39 Step 506  loss_gen_a2b:0.2919767498970032  loss_gen_b2a:0.446347713470459  loss_id_a2b:0.08538717031478882  loss_id_b2a:0.07653725147247314  A_B_A_cycle_loss:0.07284988462924957  B_A_B_cycle_loss:0.0724194347858429  loss_gen_total:3.0006396770477295  loss_dis_b:0.19602644443511963  loss_dis_a:0.2780027687549591  loss_dis_total:0.47402921319007874 
Epoch 39 Step 507  loss_gen_a2b:0.6313726902008057  loss_gen_b2a:0.4506668150424957  loss_id_a2b:0.09247271716594696  loss_id_b2a:0.08176472783088684  A_B_A_cycle_loss:0.0896764025092125  B_A_B_cycle_loss:0.08888348937034607  loss_gen_total:3.738825798034668  loss_dis_b:0.059084244072437286  loss_dis_a:0.0838795155286789  loss_dis_total:0.14296376705169678 
Epoch 39 Step 508  loss_gen_a2b:0.4278475046157837  loss_gen_b2a:0.29295241832733154  loss_id_a2b:0.07065235078334808  loss_id_b2a:0.12548622488975525  A_B_A_cycle_loss:0.11570322513580322  B_A_B_cycle_loss:0.1009816825389862  loss_gen_total:3.8683419227600098  loss_dis_b:0.105703

Epoch 39 Step 529  loss_gen_a2b:0.16602908074855804  loss_gen_b2a:0.2799704670906067  loss_id_a2b:0.0984811931848526  loss_id_b2a:0.15818293392658234  A_B_A_cycle_loss:0.1341259628534317  B_A_B_cycle_loss:0.09832468628883362  loss_gen_total:4.053826808929443  loss_dis_b:0.20171311497688293  loss_dis_a:0.13504011929035187  loss_dis_total:0.336753249168396 
Epoch 39 Step 530  loss_gen_a2b:0.5511178970336914  loss_gen_b2a:0.32915592193603516  loss_id_a2b:0.10667046904563904  loss_id_b2a:0.061532922089099884  A_B_A_cycle_loss:0.08077394962310791  B_A_B_cycle_loss:0.13894930481910706  loss_gen_total:3.9185233116149902  loss_dis_b:0.07055399566888809  loss_dis_a:0.16289854049682617  loss_dis_total:0.23345252871513367 
Epoch 39 Step 531  loss_gen_a2b:0.47062763571739197  loss_gen_b2a:0.5378969311714172  loss_id_a2b:0.1003008559346199  loss_id_b2a:0.1155068576335907  A_B_A_cycle_loss:0.1107092946767807  B_A_B_cycle_loss:0.11436425894498825  loss_gen_total:4.338298797607422  loss_dis_b:0.118470

NameError: name 'A_B_A_cycle_loss_metrics' is not defined

In [1]:
%load_ext tensorboard
%tensorboard --logdir logs/monet2photo

Reusing TensorBoard on port 6006 (pid 9964), started 1:32:51 ago. (Use '!kill 9964' to kill it.)

In [ ]:
!tensorboard dev upload --logdir logs

# Inference

# References:

1. https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix/blob/master/models/cycle_gan_model.py
2. https://hardikbansal.github.io/CycleGANBlog/
3. https://arxiv.org/abs/1703.10593
4. https://hackernoon.com/gender-and-race-change-on-your-selfie-with-neural-nets-9a9a1c9c5c16